**1. Описание и цель проекта**

В ходе проекта будет произведён анализ базы данных, содержащей информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. На основании полученных данных будут сформулированы ценностные предложения для нового продукта.

**2. Загрузка данных**

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

**3. Список таблиц БД и вывод их первых строк**

In [3]:
q = '''SELECT table_name
       FROM information_schema.tables 
       WHERE table_schema NOT IN ('information_schema','pg_catalog');'''

pd.io.sql.read_sql(q, con = engine).T

,0,1,2,3,4,5,6,7
table_name,orders,visits,advertisment_costs,publishers,authors,reviews,ratings,books


In [4]:
q = '''SELECT *  FROM books LIMIT 1'''
pd.io.sql.read_sql(q, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [5]:
q = '''SELECT *  FROM authors LIMIT 1'''
pd.io.sql.read_sql(q, con = engine)

,author_id,author
0,1,A.S. Byatt


In [6]:
q = '''SELECT *  FROM publishers LIMIT 1'''
pd.io.sql.read_sql(q, con = engine)

,publisher_id,publisher
0,1,Ace


In [7]:
q = '''SELECT *  FROM ratings LIMIT 1'''
pd.io.sql.read_sql(q, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


In [8]:
q = '''SELECT *  FROM reviews LIMIT 1'''
pd.io.sql.read_sql(q, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


<div class="alert alert-block alert-warning">
<b>Комментарии от ревьюера:</b>

Здесь всё хорошо. Ты корректно выгрузил данные и посмотрел их содержимое. Можно было бы вывести еще информацию с помощью info(). Так у нас будет информация хотя бы о размерности датасетов
</div>

**4. Ответы на вопросы исследования**

**Сколько книг вышло после 1 января 2000 года**

In [9]:
q = '''
       SELECT 
            COUNT(book_id) AS count_after_010120
       FROM 
            books
       WHERE 
            publication_date :: date > '2000-01-01'
    '''
pd.io.sql.read_sql(q, con = engine)

,count_after_010120
0,819


После начала нового тысячелетия вышла не тысяча, но 819 новых книг.

**Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [10]:
q = '''SELECT 
            books.book_id,
            books.title,
            books.author_id,
            COUNT(DISTINCT review_id) AS count_reviews,
            AVG(rating) AS mean_rating
       FROM 
            books 
            JOIN reviews ON books.book_id=reviews.book_id
            JOIN ratings ON books.book_id=ratings.book_id
       GROUP BY
            books.book_id'''

pd.io.sql.read_sql(q, con = engine)

,book_id,title,author_id,count_reviews,mean_rating
0,1,'Salem's Lot,546,2,3.666667
1,2,1 000 Places to See Before You Die,465,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,407,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,82,2,4.500000
4,5,1776,125,4,4.000000
...,...,...,...,...,...
989,996,Wyrd Sisters (Discworld #6; Witches #2),571,3,3.666667
990,997,Xenocide (Ender's Saga #3),454,3,3.400000
991,998,Year of Wonders,201,4,3.200000
992,999,You Suck (A Love Story #2),94,2,4.500000


На основании запроса сформирована таблица, содержащая информацию о количестве обзоров и средней оценке каждой книге.

**Определите издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [11]:
# используются два подзапроса - запрос ex для расчёта количества книг 
# с числом страниц более 50 по издательствам и запрос без алиаса с выборкой
# максимального значения рассчитаного количества книг
q = '''
       SELECT
           ex.publisher_id,
           ex.publisher 
       FROM
            (SELECT
                books.publisher_id,
                publisher,
                COUNT(books.book_id) cb
            FROM 
                books 
                JOIN publishers ON books.publisher_id=publishers.publisher_id
            WHERE
                books.num_pages > 50
            GROUP BY 
                books.publisher_id,
                publisher) ex
        WHERE  cb =
            (SELECT
                DISTINCT MAX(COUNT(books.book_id)) OVER()
             FROM
                books 
                JOIN publishers ON books.publisher_id=publishers.publisher_id
             WHERE
                books.num_pages > 50
             GROUP BY
                books.publisher_id,
                publisher)            
      '''

pd.io.sql.read_sql(q, con = engine)

,publisher_id,publisher
0,212,Penguin Books


Самым активным в части выпуска книг стало издательство, основанное пингвинами из мультфильма "Мадагаскар". Им помогла военная дисциплина.

**Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

Определён автор с самой высокой средней оценкой книг. Ими стали в соавторстве Дж. Роулинг и Мэри Гранпрэ за повествование о Гарри Поттере для американских читателей.

In [13]:
q = '''SELECT
           ex.author_id,
           ex.author,
           AVG(ex.ar) avg_raiting
       FROM
            (SELECT
                books.book_id,
                authors.author_id,
                authors.author,
                COUNT(ratings.book_id) cr,
                AVG(ratings.rating) ar
            FROM 
                books 
                JOIN authors ON books.author_id=authors.author_id
                JOIN ratings ON ratings.book_id=books.book_id
            GROUP BY
                books.book_id,
                authors.author_id,
                authors.author
            HAVING
                COUNT(ratings.book_id) >= 50
            ORDER BY
                ar DESC
                ) ex
        GROUP BY
            ex.author_id,
            ex.author
        ORDER BY
            avg_raiting DESC
        LIMIT
            10
      '''

pd.io.sql.read_sql(q, con = engine)

,author_id,author,avg_raiting
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,106,Dan Brown,3.754540


**Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [14]:
q = '''
       SELECT 
          DISTINCT AVG(COUNT(review_id)) OVER() avg_reviews
       FROM
           (SELECT            
               username,
               COUNT(rating_id) ratings
           FROM
               ratings
           GROUP BY
                username
           HAVING
               COUNT(rating_id) > 50) ex
            JOIN reviews ON  reviews.username = ex.username
        GROUP BY
              ex.username
              
    '''     
    
pd.io.sql.read_sql(q, con = engine)

,avg_reviews
0,24.333333


В среднем активные пользователи сформировали по 24,3 обзора.